In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

input_ = pd.read_excel('./2.subategory.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

from urllib.parse import quote
import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                jsn = '{"jsn":{"tab":"extras","groupname":"' + input_.loc[i, 'Category'] + '","subgroupname":"' + input_.loc[i, 'Subcategory'] + '","nodetype":"subgroupname"}}'

                payload = {'func': 'navnode_fetch',
                           'payload': jsn,
                           'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=payload,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['html_fill_sections']['navchildren[]']
                
                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))
                
                security_code = html.xpath('//div[@class="center"]/div')

                # = = = = = = = = = = = = = = =
                
                if len(security_code) != 0 and 'security code' in security_code[0].text:
                    continue
                    
                # = = = = = = = = = = = = = = =

                list_type = [type_.strip() for type_ in html.xpath('//td[@class="nlabel"]/a/text()')]

                # = = = = = = = = = = = = = = =
                
                list_url = ['https://www.rockauto.com'+url.strip() for url in html.xpath('//td[@class="niconspace ncollapsedicon"]/a/@href')]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                                        'No.': int(input_.loc[i, 'No.']),
                                        'Category': input_.loc[i, 'Category'],
                                        'Subcategory': input_.loc[i, 'Subcategory'],
                                        'Type': list_type,
                                        'Type_Code': [url.split(',')[-1] for url in list_url],
                                        'Url': list_url})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status != 'ok':
            df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                                     'No.': int(input_.loc[i, 'No.']),
                                     'Category': input_.loc[i, 'Category'],
                                     'Subcategory': input_.loc[i, 'Subcategory'],
                                     'Url': input_.loc[i, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Category'] + ' - ' + input_.loc[i, 'Subcategory'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['No.', 'Subcategory', 'Type'], ascending=[True, True, True])
output_correct.to_excel('./3.type-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.', 'Subcategory'], ascending=[True, True])
    output_error.to_excel('./type_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：330

Belt Drive - Tool  <->  [ok] - 剩余数量：310 - 当前时间：17:14:35
Body & Lamp Assembly - Bumper / Pad  <->  [ok] - 剩余数量：309 - 当前时间：17:14:35
Belt Drive - ATV / UTV Belt  <->  [ok] - 剩余数量：308 - 当前时间：17:14:35
Belt Drive - Pulley & Tensioner  <->  [ok] - 剩余数量：307 - 当前时间：17:14:35
Belt Drive - Refrigeration V-Belt  <->  [ok] - 剩余数量：306 - 当前时间：17:14:35
Belt Drive - Fluid / Chemical  <->  [ok] - 剩余数量：305 - 当前时间：17:14:35
Body & Lamp Assembly - Accessory Lighting  <->  [ok] - 剩余数量：304 - 当前时间：17:14:35
Belt Drive - Heavy Duty / Industrial V-Belt  <->  [ok] - 剩余数量：303 - 当前时间：17:14:35
Body & Lamp Assembly - Back Up / Reverse Lamp  <->  [ok] - 剩余数量：302 - 当前时间：17:14:35
Belt Drive - Automotive V-Belt  <->  [ok] - 剩余数量：301 - 当前时间：17:14:36
Body & Lamp Assembly - Door  <->  [ok] - 剩余数量：300 - 当前时间：17:14:36
Apparel & Gifts - RockAuto Apparel  <->  [ok] - 剩余数量：299 - 当前时间：17:14:36
Body & Lamp Assembly - Antenna  <->  [ok] - 剩余数量：298 - 当前时间：17:14:36
Body & Lamp Assembly - Cargo Management  <->  [ok] - 剩余数量：297 

Engine - Oil Cooler  <->  [ok] - 剩余数量：191 - 当前时间：17:14:47
Exhaust & Emission - Rain Cap  <->  [ok] - 剩余数量：190 - 当前时间：17:14:48
Fuel & Air - Carburetor  <->  [ok] - 剩余数量：189 - 当前时间：17:14:48
Fuel & Air - Fuel Injector  <->  [ok] - 剩余数量：188 - 当前时间：17:14:48
Fuel & Air - Filler Neck  <->  [ok] - 剩余数量：187 - 当前时间：17:14:48
Exhaust & Emission - Resonator  <->  [ok] - 剩余数量：186 - 当前时间：17:14:49
Body & Lamp Assembly - Weatherstrip  <->  [ok] - 剩余数量：185 - 当前时间：17:14:49
Fuel & Air - Fluid / Chemical  <->  [ok] - 剩余数量：184 - 当前时间：17:14:49
Fuel & Air - Fuel Line  <->  [ok] - 剩余数量：183 - 当前时间：17:14:49
Fuel & Air - Fuel Filter  <->  [ok] - 剩余数量：182 - 当前时间：17:14:49
Fuel & Air - Tool  <->  [ok] - 剩余数量：181 - 当前时间：17:14:49
Fuel & Air - Fuel Tank  <->  [ok] - 剩余数量：180 - 当前时间：17:14:50
Exhaust & Emission - Tool  <->  [ok] - 剩余数量：179 - 当前时间：17:14:50
Garage Equipment - Air Compressor  <->  [ok] - 剩余数量：178 - 当前时间：17:14:50
Garage Equipment - Axe / Hammer  <->  [ok] - 剩余数量：177 - 当前时间：17:14:50
Fuel & Air - Fuel Tank Sen

Hoses/Lines & Clamps - Water Hose  <->  [ok] - 剩余数量：69 - 当前时间：17:15:09
Heat & Air Conditioning - A/C Receiver Drier / Accumulator  <->  [ok] - 剩余数量：68 - 当前时间：17:15:10
Ignition - Electrical  <->  [ok] - 剩余数量：67 - 当前时间：17:15:10
Hoses/Lines & Clamps - Turbocharger / Intercooler Hose  <->  [ok] - 剩余数量：66 - 当前时间：17:15:10
Ignition - Tool  <->  [ok] - 剩余数量：65 - 当前时间：17:15:10
Garage Equipment - Tool Set / Storage  <->  [ok] - 剩余数量：64 - 当前时间：17:15:10
Ignition - Ignition Coil / Wiring  <->  [ok] - 剩余数量：63 - 当前时间：17:15:10
Hoses/Lines & Clamps - Power Steering Hose  <->  [ok] - 剩余数量：62 - 当前时间：17:15:11
Interior - Cargo Management  <->  [ok] - 剩余数量：61 - 当前时间：17:15:11
Interior - Cell Phone  <->  [ok] - 剩余数量：60 - 当前时间：17:15:11
Interior - Cable  <->  [ok] - 剩余数量：59 - 当前时间：17:15:11
Interior - Cup Holder  <->  [ok] - 剩余数量：58 - 当前时间：17:15:12
Interior - Cigarette Lighter  <->  [ok] - 剩余数量：57 - 当前时间：17:15:12
Body & Lamp Assembly - Overspray Protection  <->  [ok] - 剩余数量：56 - 当前时间：17:15:12
Hoses/Lines & Clamp